<a href="https://colab.research.google.com/github/herjh0405/DACON_Meal/blob/master/%EA%B5%AC%EB%82%B4%EC%8B%9D%EB%8B%B9_%EC%8B%9D%EC%88%98_%EC%9D%B8%EC%9B%90_%EC%98%88%EC%B8%A1_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DACON LH_구내식당 식수 인원 예측
* 가설 검증 및 시각화

## 환경 설정

### Model Install

In [ ]:
!pip install pycaret
!pip install konlpy

### Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import Library

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
np.random.seed(0)

from pycaret.regression import *
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, log_loss

from tqdm.notebook import tqdm
from konlpy.tag import Kkma

import torch
import os, re

In [ ]:
# 한글 폰트 사용
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os

def change_matplotlib_font(font_download_url):
    FONT_PATH = 'MY_FONT'
    
    font_download_cmd = f"wget {font_download_url} -O {FONT_PATH}.zip"
    unzip_cmd = f"unzip -o {FONT_PATH}.zip -d {FONT_PATH}"
    os.system(font_download_cmd)
    os.system(unzip_cmd)
    
    font_files = fm.findSystemFonts(fontpaths=FONT_PATH)
    for font_file in font_files:
        fm.fontManager.addfont(font_file)

    font_name = fm.FontProperties(fname=font_files[0]).get_name()
    matplotlib.rc('font', family=font_name)
    print("font family: ", plt.rcParams['font.family'])

font_download_url = "https://fonts.google.com/download?family=Noto%20Sans%20KR"
change_matplotlib_font(font_download_url)

## Data Preprocessing

### Load Data

In [ ]:
path = '/content/drive/MyDrive/DACON/Dacon_Industry_Meal/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')

train.columns = ['일자', '요일', '정원','휴가자', '출장자', '야근자',\
                 '재택근무자', '조식', '중식', '석식', '중식계', '석식계']
test.columns = ['일자', '요일', '정원','휴가자', '출장자', '야근자',\
                 '재택근무자', '조식', '중식', '석식']

In [ ]:
train[['재택근무자', '중식계', '석식계']] = train[['재택근무자', '중식계', '석식계']].astype('int')
test['재택근무자'] = test['재택근무자'].astype('int')

def df_date(df) :
    df['일자'] = pd.to_datetime(df['일자'])
    df['년'] = df['일자'].dt.year
    df['월'] = df['일자'].dt.month
    df['일'] = df['일자'].dt.day
    df['월일'] = df['일자'].apply(lambda x : str(x)[5:10])

    return df

train = df_date(train)
test = df_date(test)

## 가설 수립 및 검정

### 가설 1. 재택근무는 코로나 때문에 생겼을 것이다.
* 이후에, 코로나 확진자 수와 연동할 것이지만, 현재로써도 가설이 맞음을 확인할 수 있다.
![image](https://user-images.githubusercontent.com/54921730/120911062-14813f80-c6bf-11eb-85e6-41bd263e1eb2.png)
* 이제 여러가지 의문점이 추가로 등장
    * 재택 근무자 수가 증가함에 따라 식수 인원도 증가하는가?
        * 의외의 결과 발생 -> 코로나 환자 증가로 인해, 외식이 어려워지면서 식수 인원이 증가한 것으로 추측
            * 그렇다면 감시 단계 데이터도 연동해야하는가?  
![image](https://user-images.githubusercontent.com/54921730/120911123-8e192d80-c6bf-11eb-9146-1a52356cf9d1.png)
![image](https://user-images.githubusercontent.com/54921730/120911048-f87d9e00-c6be-11eb-9f3c-85d39d82a74d.png)
    * 그렇다면 코로나 전후로 데이터를 구분해야하는 것은 아닌가?

### 결론
* 위 여러 의문점들이 남아있지만, 아직 명쾌한 해답이 나오지 않았음.
* 컴퓨터가 학습하기 너무 어려운 데이터라 판단, 데이터에서 제외

In [ ]:
# 코로나 발생현황 데이터 불러와서 비교할 예정
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.parse import unquote, quote_plus, urlencode

# 1. URL 파라미터 분리하기.
# Service URL
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
servicekey = '{9qhUwKCc9ASN8d7C8HnGWLyQOr1P4rkXKxsYwShxM6PLhVSnNwqlkZQsY%2Fk%2Bx4Rixmyznbm65TySlJtMbVi3KA%3D%3D}'
decodekey = unquote(servicekey)

queryParams = '?' + urlencode(    # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것이다. 메타코드 아님.
    {
        quote_plus('ServiceKey') : servicekey,    # 필수 항목 1 : 서비스키 (본인의 서비스키)
    }
)

URL = url + queryParams
request = Request(URL)
page = urlopen(request).read()
soup = BeautifulSoup(page, 'html.parser')

In [ ]:
plt.plot(train['일자'], train['재택근무자'])
plt.xlabel('일자')
plt.ylabel('재택 근무자 수')
plt.title('재택 근무자 수 추이')
plt.show()

In [ ]:
train_18 = train.loc[train['년']==2018]
train_19 = train.loc[train['년']==2019]
train_20 = train.loc[train['년']==2020]

In [ ]:


plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplot(121)
bar_width = 0.3
index = train_19.groupby(['월'])['중식계'].sum().index
alpha = 0.5

plt.bar(index-bar_width, train_18.groupby(['월'])['중식계'].sum(),
        label='2018', width=0.3, alpha=alpha)
plt.bar(index, train_19.groupby(['월'])['중식계'].sum(), label='2019', width=0.3, alpha=alpha)
plt.bar(index+bar_width,
        train_20.groupby(['월'])['중식계'].sum(), label='2020', width=0.3, alpha=alpha)
plt.bar(index+bar_width,
        train_20.groupby(['월'])['재택근무자'].sum(), label='재택근무자',
        bottom=train_20.groupby(['월'])['중식계'].sum(), width=0.3, alpha=alpha)

plt.xticks(index, label=index)
plt.xlabel('월')
plt.ylabel('중식계 총합')
plt.legend('best', labels=['2018', '2019', '2020', '재택근무자'])
plt.title('코로나가 중식계에 끼친 영향', fontsize=12)

plt.subplot(122)
bar_width = 0.3
index = train_19.groupby(['월'])['석식계'].sum().index
alpha = 0.5

plt.bar(index-bar_width, train_18.groupby(['월'])['석식계'].sum(),
        label='2018', width=0.3, alpha=alpha)
plt.bar(index, train_19.groupby(['월'])['석식계'].sum(), label='2019', width=0.3, alpha=alpha)
plt.bar(index+bar_width,
        train_20.groupby(['월'])['석식계'].sum(), label='2020', width=0.3, alpha=alpha)
plt.bar(index+bar_width,
        train_20.groupby(['월'])['재택근무자'].sum(), label='재택근무자',
        bottom=train_20.groupby(['월'])['석식계'].sum(), width=0.3, alpha=alpha)

plt.xticks(index, label=index)
plt.xlabel('월')
plt.ylabel('석식계 총합')
plt.legend('best', labels=['2018', '2019', '2020', '재택근무자'])
plt.title('코로나가 석식계에 끼친 영향', fontsize=12)
plt.show()